# **資料準備和特徵工程**

資料對於訓練和準備模型至關重要。 在本 Notebook 中，我們將介紹如何將資料載入到 ML.NET 中並確保其格式正確，以便 ML.NET 可以使用它。

在本 Notebook 中，您將學習如何...

- 將資料載入到 ML.NET
- 應用資料轉換來幫助 ML.NET 理解資料

## **ML.NET載入資料**

### 什麼是 IDataView？

[IDataView](https://docs.microsoft.com/dotnet/api/microsoft.ml.idataview?view=ml-dotnet) 是 ML.NET 為訓練載入的資料格式。 它是一組介面和元件，可為機器學習和高階分析應用程式提供模式化資料的高效組合處理。 它旨在優雅高效地處理高維資料和大型資料集。

IDataView 具有一般模式支援，因為檢視可以有任意數量的列，每列都有關聯的名稱、索引、資料型別和可選註解。

### **如何建立 IDataView**

您可以使用任何載入資料的方法來建立 IDataView：

- 文字載入器
- LoadFromEnumerable
- 資料庫載入器
- LoadFromTextFile

有關更多文件和範例，請參閱[從檔案和其他來源載入資料](https://docs.microsoft.com/dotnet/machine-learning/how-to-guides/load-data-ml-net)。

In [1]:
#i "nuget:https://pkgs.dev.azure.com/dnceng/public/_packaging/MachineLearning/nuget/v3/index.json"
#r "nuget: Microsoft.ML, 2.0.0-preview.22356.1"

Installed Packages Microsoft.ML, 2.0.0-preview.22313.1

In [1]:
using Microsoft.ML;
using Microsoft.ML.Data;
using Microsoft.ML.Transforms; 

## **下載或查詢資料**
以下程式碼嘗試在幾個已知位置找到資料檔案，或者它將從已知的 GitHub 位置下載它。

In [1]:
using System;
using System.IO;
using System.Net;

string EnsureDataSetDownloaded(string fileName)
{

	// This is the path if the repo has been checked out.
	var filePath = Path.Combine(Directory.GetCurrentDirectory(),"data", fileName);

	if (!File.Exists(filePath))
	{
		// This is the path if the file has already been downloaded.
		filePath = Path.Combine(Directory.GetCurrentDirectory(), fileName);
	}

	if (!File.Exists(filePath))
	{
		using (var client = new WebClient())
		{
			client.DownloadFile($"https://raw.githubusercontent.com/dotnet/csharp-notebooks/main/machine-learning/data/{fileName}", filePath);
		}
		Console.WriteLine($"Downloaded {fileName}  to : {filePath}");
	}
	else
	{
		Console.WriteLine($"{fileName} found here: {filePath}");
	}

	return filePath;
}

#### 從檔案載入

[TextLoader](https://docs.microsoft.com/dotnet/api/microsoft.ml.data.textloader?view=ml-dotnet) 可以將結構化檔案載入到 IDataView 中。 結構化訊息表示為數據的列和行。

IDataView 具有一般模式支援，因為檢視可以有任意數量的列，每列都有關聯的名稱、索引、資料型別和可選註解。 您可以使用 Plain-Old-CLR-Objects (POCO) 或類為您的資料定義架構。

關於 ModelInput 類別的一些注意事項。
- `LoadColumn` 屬性指定列索引。 這是從檔案載入時的必要屬性。
- `ColumnName` 屬性用於將列的名稱設定為屬性名稱以外的名稱。 記憶體中的物件使用屬性名稱。 但是，對於資料處理和建立機器學習模型，ML.NET 會使用 ColumnName 屬性中提供的值覆蓋和引用該屬性。

In [1]:
public class ModelInput
{
    [LoadColumn(0)]
    [ColumnName(@"vendor_id")]
    public string Vendor_id { get; set; }

    [LoadColumn(1)]
    [ColumnName(@"rate_code")]
    public float Rate_code { get; set; }

    [LoadColumn(2)]
    [ColumnName(@"passenger_count")]
    public float Passenger_count { get; set; }

    [LoadColumn(3)]
    [ColumnName(@"trip_time_in_secs")]
    public float Trip_time_in_secs { get; set; }

    [LoadColumn(4)]
    [ColumnName(@"trip_distance")]
    public float Trip_distance { get; set; }

    [LoadColumn(5)]
    [ColumnName(@"payment_type")]
    public string Payment_type { get; set; }

    [LoadColumn(6)]
    [ColumnName(@"fare_amount")]
    public float Fare_amount { get; set; }

}

所有 ML.NET 操作都從 [MLContext](https://docs.microsoft.com/dotnet/api/microsoft.ml.mlcontext) 類開始。 初始化 mlContext 會建立一個新的 ML.NET 環境，該環境可以在模型建立工作流物件之間共享。 它在概念上類似於實體框架中的 DBContext。

In [1]:
//Create MLContext
MLContext mlContext = new MLContext();

根據 ModelInput 型別建立一個 [TextLoader](https://docs.microsoft.com/dotnet/api/microsoft.ml.data.textloader?view=ml-dotnet)。 然後使用文字載入器從資料檔案中載入。 至少，載入程式需要被告知檔案是否有標題，以及檔案使用的分隔符。

您也可以使用直接方法 [LoadFromTextFile](https://docs.microsoft.com/dotnet/api/microsoft.ml.textloadersavercatalog.loadfromtextfile?view=ml-dotnet)。 TextLoader 的優點是它提供了從不同位置的多個檔案載入檔案的選項。

In [1]:
var trainDataPath = EnsureDataSetDownloaded("taxi-fare.csv");

// Create TextLoader based on the Model Input type. 
TextLoader textLoader = mlContext.Data.CreateTextLoader<ModelInput>(separatorChar: ',', hasHeader: true);

// Load the data into an IDataView. Load() method can support multiple files. 
// Files must they have the same separator character, header, column names, etc. 
IDataView data = textLoader.Load(trainDataPath);

data.Preview(1); 

#### **在記憶體集合中載入**

ML.NET 支援從記憶體集合中載入資料。 這使得使用 C# 從 JSON 或 XML 檔案載入變得容易。 瞭解如何 [使用 C# 反序列化 JSON](https://docs.microsoft.com/dotnet/standard/serialization/system-text-json-how-to?pivots=dotnet-6-0#how-to-read-json-as-net-objects-deserialize) 或使用 [XML 序列化程式](https://docs.microsoft.com/dotnet/api/system.xml.serialization?view=net-6.0) 將這些檔案放入記憶體。

在記憶體中收集資料後，您可以使用 [LoadFromEnumerable](https://docs.microsoft.com/dotnet/api/microsoft.ml.dataoperationscatalog.loadfromenumerable?view=ml-dotnet) 將其載入到 ML.NET ） 方法。

In [1]:
ModelInput[] inMemoryCollection = new ModelInput[]
{
    new ModelInput
    {
        Vendor_id = "CMT",
        Rate_code = 1,
        Passenger_count = 1,
        Trip_time_in_secs = 1271,
        Trip_distance = 3.8f,
        Payment_type = "CRD",
        Fare_amount = 17.5f,
    },
    new ModelInput
    {
        Vendor_id = "VST",
        // missing Rate_code
        Passenger_count = 1,
        Trip_time_in_secs = 474,
        Trip_distance = 1.5f,
        Payment_type = "CSH",
        Fare_amount = 8, 
    }
};

In [1]:
// Create MLContext
MLContext mlContext = new MLContext();

//Load Data
IDataView data = mlContext.Data.LoadFromEnumerable<ModelInput>(inMemoryCollection);

data.Preview(1);

### **DataFrame 和 IDataView 有什麼區別？**

您可能聽說過 [DataFrame](https://docs.microsoft.com/dotnet/api/microsoft.data.analysis.dataframe?view=ml-dotnet-preview) 型別。它是另一種用於載入、檢視和操作筆記型電腦常用資料的工具。它實現了一個 IDataView，因此可以輕鬆地將其傳遞給 ML.NET。

DataFrame 和 IDataView 在某種意義上非常相似，它們都是以表格格式表示資料併為其應用轉換的方式。一些關鍵區別：

- DataFrame 僅支援載入分隔檔案。
- DataFrame 在記憶體上執行，因此受限於 PC 上的記憶體量。

在對資料樣本執行探索性資料分析等任務時，建議使用 DataFrame。檢視參考筆記本 [REF-資料處理](https://github.com/dotnet/csharp-notebooks/blob/main/machine-learning/REF-Data%20Processing.ipynb) 中使用 Data Frames 的範例操作數據檔案進行訓練。

建議使用 IDataView 對較大的資料集進行訓練，以及本筆記本中的範例所使用的內容。在這種情況下，較大的資料集被定義為無法放入記憶體的資料集。

## **資料轉換**

ML.NET 支援多種資料轉換，可將資料轉換為所需格式並幫助您更正資料。 一些常見的操作是操作列、規範化值、替換缺失值、轉換值等等。

有關詳細訊息，請參閱 [資料轉換](https://docs.microsoft.com/dotnet/machine-learning/resources/transforms)。

以下是一些常見的轉換。

### **分類資料**

One Hot 編碼是對包含類別的資料的重要轉換。 ML 演算法要求資料是數字的，它不知道如何處理表示類別的字串。 vendor_id 和 payment_type 列是分類的，vendor 可以是『CMD』或『VST』，payment 可以是『CReDit』或『CaSH』。 One Hot  採用傳入的字串值並將它們轉換為數字資料。

In [1]:
var pipeline = mlContext.Transforms.Categorical.OneHotEncoding(
    new[] 
    { new InputOutputColumnPair(@"vendor_id"), 
    new InputOutputColumnPair(@"payment_type")},
    OneHotEncodingEstimator.OutputKind.Binary); 

讓我們在 vendor_id 和 payment_type 上測試上述轉換。 結果應該是每個類別的向量值。 對於 Vendor_Id，CMT 變為『000』，VST 變為『001』。 我們將為新轉換的型別建立一個新的 ModelInputTransformed 類。

In [1]:
using System.Numerics; 

public class ModelInputTransformed
{
    [LoadColumn(0)]
    [ColumnName(@"vendor_id")]
    public VBuffer<Single> Vendor_id { get; set; }

    [LoadColumn(1)]
    [ColumnName(@"rate_code")]
    public float Rate_code { get; set; }

    [LoadColumn(2)]
    [ColumnName(@"passenger_count")]
    public float Passenger_count { get; set; }

    [LoadColumn(3)]
    [ColumnName(@"trip_time_in_secs")]
    public float Trip_time_in_secs { get; set; }

    [LoadColumn(4)]
    [ColumnName(@"trip_distance")]
    public float Trip_distance { get; set; }

    [LoadColumn(5)]
    [ColumnName(@"payment_type")]
    public VBuffer<Single> Payment_type { get; set; }

    [LoadColumn(6)]
    [ColumnName(@"fare_amount")]
    public float Fare_amount { get; set; }
}

In [1]:
// Run the transform
IDataView transformedData = pipeline.Fit(data).Transform(data);
var convertedData = mlContext.Data.CreateEnumerable<ModelInputTransformed>(transformedData, true);

// One Hot Encoding of two columns 'vendor_id' and 'payment_type'.
Console.WriteLine("Vendor_Id" +"\t" + "Payment_Type"); 
foreach (ModelInputTransformed item in convertedData)
{    
    Console.WriteLine("{0}\t\t{1}", string.Join(" ", item.Vendor_id.DenseValues()),
                    string.Join(" ", item.Payment_type.DenseValues()));
}

0 0 1		0 0 1


### **替換缺失值**

另一種常見的操作是替換缺失值。 這裡我們使用預設替換模式，將值替換為其型別的預設值。

In [1]:
pipeline.Append(mlContext.Transforms.ReplaceMissingValues(
    new[] { new InputOutputColumnPair(@"rate_code", @"rate_code"), 
    new InputOutputColumnPair(@"passenger_count", @"passenger_count"), 
    new InputOutputColumnPair(@"trip_time_in_secs", @"trip_time_in_secs"), 
    new InputOutputColumnPair(@"trip_distance", @"trip_distance") })); 

讓我們再執行轉換並檢視填充的值。 我們缺少第二個虛擬物件的 rate_code。

In [1]:
IDataView transformedData = pipeline.Fit(data).Transform(data);
var convertedData = mlContext.Data.CreateEnumerable<ModelInputTransformed>(transformedData, true);

"Rate_code: " + convertedData.ElementAt(1).Rate_code

Rate_code: 0

現在讓我們將所有特徵列連線成一個向量列。 許多 ML 訓練器期望特徵是向量型別，因為將操作應用於向量更有效。

In [1]:
pipeline.Append(mlContext.Transforms.Concatenate(
    @"Features", new[] { @"vendor_id", @"payment_type", @"rate_code", @"passenger_count", @"trip_time_in_secs", @"trip_distance" }));

我們現在有一個載入的 IDataView 和通道用於訓練。

# **進階學習**

> [⏩ 下一個學習模組 - 訓練 and AutoML](https://github.com/doggy8088/csharp-notebooks/blob/zh-tw/machine-learning/03-Training%20and%20AutoML.ipynb)  
> [⏪ 上一個學習模組 - Intro to Machine Learning](https://github.com/doggy8088/csharp-notebooks/blob/zh-tw/machine-learning/01-Intro%20to%20Machine%20Learning.ipynb)  